In [1]:
import socket
import struct
import threading
import time
import sys

class listenThread (threading.Thread):
    def __init__(self, threadID, name, socket, freq):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.sock = socket
        self.freq = freq
        
    def run(self):
        print 'Thread {} "{}" Started'.format(self.threadID, self.name)
        while(1):
            time.sleep(self.freq)
            try:
                message, address = self.sock.recvfrom(10240)
                
                print '{}, {}, Message: {}, Sender: {}'.format(time.ctime(time.time()), self.name, message, address)
            except:
                None
                #print "Unexpected error:", sys.exc_info()[0]

class heartBeatThread (threading.Thread):
    def __init__(self, threadID, name, socket, freq, mcast_grp, mcast_port):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.sock = socket
        self.freq = freq
        self.mcast_grp = mcast_grp
        self.mcast_port = mcast_port
        
    def run(self):
        print 'Thread {} "{}" Started'.format(self.threadID, self.name)
        while(1):
            time.sleep(self.freq)
            try:
                self.sock.sendto("HeartBeat", (self.mcast_grp, self.mcast_port))
                print '{}: ****** Heartbeat ******'.format(time.ctime(time.time()), self.name)
            except:
                None
                #print "Unexpected error:", sys.exc_info()[0]

# Multicast Socket setup - Receive 
MCAST_GRP1 = '224.1.1.1'
MCAST_PORT1 = 5007
sock1 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock1.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock1.bind(('', MCAST_PORT1))  # use MCAST_GRP instead of '' to listen only
                             # to MCAST_GRP, not all groups on MCAST_PORT
mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP1), socket.INADDR_ANY)
sock1.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)
sock1.setblocking(0)

# Multicast Socket setup - Send 
MCAST_GRP2 = '224.1.1.1' # despite binding to a group we end up accepting all multicast messages in the listen
MCAST_PORT2 = 5008
sock2 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock2.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, 2)

# Create new threads
thread1 = listenThread(1, "Listen", sock1, 0.05)
thread2 = heartBeatThread(2, "HeartBeat", sock2, 5, MCAST_GRP2, MCAST_PORT2)

# Start new Threads
thread1.start()
thread2.start()

while(1):
    time.sleep(100)


Thread 1 "Listen" Started
Thread 2 "HeartBeat" Started
Mon Jul  2 02:55:53 2018: ****** Heartbeat ******
Mon Jul  2 02:55:58 2018: ****** Heartbeat ******
Mon Jul  2 02:55:58 2018, Listen, Message: Slave 1 Alive, Sender: ('172.17.0.7', 40318)
Mon Jul  2 02:56:03 2018: ****** Heartbeat ******
Mon Jul  2 02:56:03 2018, Listen, Message: Slave 1 Alive, Sender: ('172.17.0.7', 40318)
Mon Jul  2 02:56:08 2018: ****** Heartbeat ******
Mon Jul  2 02:56:08 2018, Listen, Message: Slave 1 Alive, Sender: ('172.17.0.7', 40318)
Mon Jul  2 02:56:08 2018, Listen, Message: Slave 2 Alive, Sender: ('172.17.0.7', 50153)
Mon Jul  2 02:56:13 2018: ****** Heartbeat ******
Mon Jul  2 02:56:13 2018, Listen, Message: Slave 1 Alive, Sender: ('172.17.0.7', 40318)
Mon Jul  2 02:56:13 2018, Listen, Message: Slave 2 Alive, Sender: ('172.17.0.7', 50153)
Mon Jul  2 02:56:18 2018: ****** Heartbeat ******
Mon Jul  2 02:56:18 2018, Listen, Message: Slave 1 Alive, Sender: ('172.17.0.7', 40318)
Mon Jul  2 02:56:18 2018, Lis

KeyboardInterrupt: 